In [20]:
import math
import json

import requests
from bs4 import BeautifulSoup

from stem import Signal
from stem.control import Controller

from fake_useragent import UserAgent

from tqdm.notebook import tqdm

In [21]:
def get_request(url):
    renew_tor_ip()
    
    user_agent = UserAgent().chrome
    headers = {'User-Agent': user_agent}
    
    
    session = requests.session()

    session.proxies = {}
    session.proxies['http']='socks5h://localhost:9050'
    session.proxies['https']='socks5h://localhost:9050'

    try:
        r = session.get(url, headers=headers)
    except Exception as e:
        print(str(e))
    else:
        return r   

def get_current_ip():
    session = requests.session()

    # TO Request URL with SOCKS over TOR
    session.proxies = {}
    session.proxies['http']='socks5h://localhost:9050'
    session.proxies['https']='socks5h://localhost:9050'

    try:
        r = session.get('http://httpbin.org/ip')
    except Exception as e:
        print(str(e))
    else:
        return r.text.split()[2].replace('\"', '')


def renew_tor_ip():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate(password="tor_for_parsing")
        controller.signal(Signal.NEWNYM)

In [22]:
total_cars = 3350
cars_per_page = 38
pages_with_cars = math.ceil(total_cars/cars_per_page)

In [23]:
pages_with_cars

89

In [24]:
auto_ru_url = 'https://auto.ru/cars/volkswagen/polo/used/?sort=price-asc&page={}'

In [25]:
20 % 10

0

In [26]:
def parse_auto_ru(number_of_pages, links):
    range_list = list(range(1, number_of_pages + 1))
    for page in tqdm(range_list):
        
        prev_links = set(links).copy()
        
        get_page(auto_ru_url, page, links)
        
        if prev_links == set(links):
            i = 5
            while prev_links == set(links) and i > 0:
                get_page(auto_ru_url, page, links)
                i -= 1
        
        print(f'page: {page}/{number_of_pages}, links collected: {len(links)}, unique_links: {len(set(links))}')
        
        if page & 10 == 0:
            with open('volk_polo_pages.json', 'r+') as file:
                file.seek(0)
                file.truncate()
                json.dump(links, file)

def get_page(main_url, page_num, links):
    try:
        resp = get_request(main_url.format(page_num))
    except Exception:
        return None
    else:
        if resp:
            if resp.status_code == 200:
                get_cars_on_page(links, resp.text)

def get_cars_on_page(links, content):
    soup = BeautifulSoup(content, 'html.parser')
    cars_on_page = soup.find_all('div', {'class': 'ListingItem-module__thumb'})

    for i, car in enumerate(cars_on_page):
        link_to_car = car.find('a', {'class': 'Link OfferThumb'})
        if link_to_car:
            link_to_car = link_to_car.get('href', None)
            if link_to_car:
                links.append(link_to_car)

In [27]:
cars_links=[]

In [28]:
parse_auto_ru(pages_with_cars, cars_links)

page: 1/89, links collected: 37, unique_links: 37
page: 2/89, links collected: 74, unique_links: 74
page: 3/89, links collected: 111, unique_links: 111
page: 4/89, links collected: 148, unique_links: 148
page: 5/89, links collected: 185, unique_links: 185
page: 6/89, links collected: 222, unique_links: 222
page: 7/89, links collected: 259, unique_links: 259
page: 8/89, links collected: 296, unique_links: 296
page: 9/89, links collected: 333, unique_links: 333
page: 10/89, links collected: 370, unique_links: 370
page: 11/89, links collected: 407, unique_links: 407
page: 12/89, links collected: 444, unique_links: 444
page: 13/89, links collected: 481, unique_links: 481
page: 14/89, links collected: 518, unique_links: 518
page: 15/89, links collected: 555, unique_links: 555
page: 16/89, links collected: 592, unique_links: 592
page: 17/89, links collected: 629, unique_links: 629
page: 18/89, links collected: 666, unique_links: 666
page: 19/89, links collected: 703, unique_links: 703
page: 

In [30]:
len(list(set(cars_links)))

2976

In [34]:
first_1800_pages = cars_links.copy()

In [31]:
with open('volk_polo_used_pages.json', 'w') as file:
    file.seek(0)
    file.truncate()
    json.dump(list(set(cars_links)), file)